In [1]:
# Cell 1: ultralytics 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [2]:
# Cell 2: Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cell 3: 추론 대상 압축 해제
import os, zipfile

zip_path = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/unlabeled_seatbelt"
unzip_output = "/content/unlabeled_infer"
os.makedirs(unzip_output, exist_ok=True)

# zip 파일 하나 혹은 여러 개 모두 해제
for fname in os.listdir(zip_path):
    if fname.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(zip_path, fname), 'r') as zip_ref:
            zip_ref.extractall(unzip_output)

print(f"✅ 압축 해제 완료: {unzip_output}")

✅ 압축 해제 완료: /content/unlabeled_infer


In [4]:
import os
from glob import glob

# 압축 풀린 루트 확인
print("루트 폴더 내 디렉토리:")
print(os.listdir("/content/unlabeled_infer"))

# 이미지 경로 확인 (재귀적으로 탐색)
img_files = glob("/content/unlabeled_infer/**/*.jpg", recursive=True)
print(f"총 이미지 수: {len(img_files)}")

# 상위 몇 개만 출력
for path in img_files[:5]:
    print(path)

루트 폴더 내 디렉토리:
['train.txt', 'obj.names', 'obj_train_data', 'obj.data']
총 이미지 수: 10000
/content/unlabeled_infer/obj_train_data/20240524_155619_NOR_0024_0.jpg
/content/unlabeled_infer/obj_train_data/20240525_185359_NOR_0126_0.jpg
/content/unlabeled_infer/obj_train_data/20240524_133004_NOR_0058_0.jpg
/content/unlabeled_infer/obj_train_data/20240524_164324_NOR_0007_0.jpg
/content/unlabeled_infer/obj_train_data/20240524_175101_NOR_0029_0.jpg


In [5]:
import os
import shutil
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import cv2

# 1. 설정
model_path = "/content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/best.pt"
img_input_dir = "/content/unlabeled_infer/obj_train_data"  # 추론 대상 원본 이미지 폴더
save_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result"  # ✅ 저장 위치 (Drive)

# 폴더 생성
os.makedirs(save_dir, exist_ok=True)

# 2. 모델 로드
model = YOLO(model_path)

# 비정상 이미지 정의 및 이동
invalid_img_dir = "/content/invalid_images"
os.makedirs(invalid_img_dir, exist_ok=True)

valid_img_files = []
invalid_img_files = []

print("🔍 Checking images for validity...")
for img_path in tqdm(img_files):
    try:
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError("이미지를 불러올 수 없습니다")
        h, w = img.shape[:2]
        if w < 10 or h < 10:
            raise ValueError(f"너무 작은 이미지: {w}x{h}")
        # 유효
        valid_img_files.append(img_path)
    except Exception as e:
        print(f"⚠️ Skipping invalid image: {img_path} → {e}")
        invalid_img_files.append(img_path)
        # 추론 디렉토리에서 제거
        try:
            shutil.move(img_path, os.path.join(invalid_img_dir, os.path.basename(img_path)))
        except Exception as e2:
            print(f"❌ 이동 실패: {img_path} → {e2}")

print(f"\n✅ 유효 이미지 수: {len(valid_img_files)}")
print(f"🚫 비정상 이미지 수: {len(invalid_img_files)} (이미 추론 디렉토리에서 이동됨)")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔍 Checking images for validity...


100%|██████████| 10000/10000 [00:16<00:00, 606.07it/s]


✅ 유효 이미지 수: 10000
🚫 비정상 이미지 수: 0 (이미 추론 디렉토리에서 이동됨)


In [6]:
# 4. 추론 실행
results = model.predict(
    source=img_input_dir, # Use the list of valid images
    conf=0.5,
    save_txt=True,
    # save_conf=True,
    save=True,
    imgsz=640,
    project=save_dir,
    name="seatbelt_yolo_export",
    # exist_ok=True
    # stream=True
)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image 5004/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0062_0.jpg: 352x640 1 Seatbelt, 141.6ms
image 5005/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0063_0.jpg: 352x640 1 Seatbelt, 142.6ms
image 5006/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0064_0.jpg: 384x640 1 Seatbelt, 161.0ms
image 5007/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0065_0.jpg: 384x640 1 Seatbelt, 155.5ms
image 5008/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0066_0.jpg: 352x640 1 Seatbelt, 155.3ms
image 5009/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0067_0.jpg: 352x640 1 Seatbelt, 144.2ms
image 5010/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0068_0.jpg: 384x640 1 Seatbelt, 158.4ms
image 5011/10000 /content/unlabeled_infer/obj_train_data/20240524_163941_NOR_0069_0.jpg: 352x640 1 Seatbelt, 144.4ms
image 5012/10000 /content/un

In [7]:
# Cell 3: 누락된 .txt 라벨 자동 생성
import glob

result_dir = os.path.join(save_dir, "seatbelt_yolo_export")
label_dir = os.path.join(result_dir, "labels")
image_dir = result_dir

# 이미지 리스트 수집
img_files = glob.glob(os.path.join(image_dir, "*.jpg"))

# 누락된 라벨 생성
for img_path in img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")
    if not os.path.exists(txt_path):
        open(txt_path, 'w').close()

print("✅ 누락된 .txt 파일 생성 완료")

✅ 누락된 .txt 파일 생성 완료


In [8]:
from glob import glob
from PIL import Image
from collections import Counter
import numpy as np

# 🔍 이미지 폴더 경로 설정 (필요시 수정)
image_dir = "/content/unlabeled_infer/obj_train_data"

# 🔍 지원되는 이미지 확장자
img_exts = ['.jpg', '.jpeg', '.png']

# 이미지 파일 전체 수집
img_files = []
for ext in img_exts:
    img_files.extend(glob(f"{image_dir}/**/*{ext}", recursive=True))

# 이미지 해상도 수집
sizes = []
for path in img_files:
    try:
        with Image.open(path) as img:
            sizes.append(img.size)  # (width, height)
    except:
        print(f"⚠️ 이미지 열기 실패: {path}")

# 🎯 통계 출력
print(f"총 이미지 수: {len(sizes)}")

# 가장 흔한 해상도 top 5
common_sizes = Counter(sizes).most_common(5)
print("\n📊 가장 흔한 해상도 top 5:")
for size, count in common_sizes:
    print(f"{size[0]}x{size[1]} → {count}장")

# 평균 해상도
widths, heights = zip(*sizes)
avg_w, avg_h = np.mean(widths), np.mean(heights)
print(f"\n📏 평균 해상도: {int(avg_w)} x {int(avg_h)}")

총 이미지 수: 10000

📊 가장 흔한 해상도 top 5:
398x235 → 69장
424x249 → 66장
427x238 → 66장
426x244 → 65장
408x237 → 63장

📏 평균 해상도: 475 x 252


In [9]:
# Cell X: YOLO 라벨 경계 확장 후처리 (비율 기준)
import os
from glob import glob

# 기존 YOLO 라벨 폴더
label_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/labels"
# 수정된 라벨 저장 폴더
output_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/labels_corrected"
os.makedirs(output_dir, exist_ok=True)

margin = 0.008  # 300px 기준 기댓값 2px

def clamp(x):
    return max(0.0, min(1.0, x))

for txt_file in glob(os.path.join(label_dir, "*.txt")):
    new_lines = []
    with open(txt_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls, x_c, y_c, w, h = map(float, parts)

            # YOLO → 좌표 변환
            x1 = x_c - w / 2
            x2 = x_c + w / 2
            y1 = y_c - h / 2
            y2 = y_c + h / 2

            # 경계 확장 적용
            if x1 < margin: x1 = 0.0
            if x2 > 1 - margin: x2 = 1.0
            if y1 < margin: y1 = 0.0
            if y2 > 1 - margin: y2 = 1.0

            # 다시 YOLO 포맷으로
            new_xc = (x1 + x2) / 2
            new_yc = (y1 + y2) / 2
            new_w = x2 - x1
            new_h = y2 - y1

            new_lines.append(f"{int(cls)} {clamp(new_xc):.6f} {clamp(new_yc):.6f} {clamp(new_w):.6f} {clamp(new_h):.6f}\n")

    out_path = os.path.join(output_dir, os.path.basename(txt_file))
    with open(out_path, 'w') as f:
        f.writelines(new_lines)

print("✅ YOLO 라벨 경계 확장 후처리 완료 (정규화 비율 기준)") # 대략 3분 소요

✅ YOLO 라벨 경계 확장 후처리 완료 (정규화 비율 기준)


In [10]:
# ✅ 비정상 이미지(.jpg)에도 빈 .txt 생성
# result_dir 및 label_dir는 이미 이전 셀에서 정의됨
invalid_img_dir = "/content/invalid_images"
output_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/labels_corrected"

# 비정상 이미지 리스트 수집
invalid_imgs = glob(os.path.join(invalid_img_dir, "*.jpg"))

# 빈 .txt 생성
for img_path in invalid_imgs:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(output_dir, base + ".txt")
    if not os.path.exists(txt_path):
        open(txt_path, 'w').close()

print(f"🗂️ 비정상 이미지용 .txt {len(invalid_imgs)}개 생성 완료")


🗂️ 비정상 이미지용 .txt 0개 생성 완료


In [11]:
from shutil import make_archive
import shutil
import os
from glob import glob

# YOLO 추론 결과 디렉토리
result_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export"
label_dir = os.path.join(result_dir, "labels_corrected")
image_dir = result_dir
invalid_img_dir = "/content/invalid_images"

# export zip 임시 디렉토리 생성
export_dir = os.path.join(result_dir, "export_zip_temp")
os.makedirs(export_dir, exist_ok=True)

# 이미지 리스트: 추론 이미지 + 비정상 이미지 포함
inferred_img_files = glob(os.path.join(image_dir, "*.jpg"))
invalid_img_files = glob(os.path.join(invalid_img_dir, "*.jpg"))
total_img_files = inferred_img_files + invalid_img_files

print(f"📦 압축 대상 총 이미지 수: {len(total_img_files)}")

# 이미지 + 라벨 복사
for img_path in total_img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    dst_img = os.path.join(export_dir, base + ".jpg")
    shutil.copy(img_path, dst_img)

    txt_path = os.path.join(label_dir, base + ".txt")
    if os.path.exists(txt_path):
        shutil.copy(txt_path, os.path.join(export_dir, base + ".txt"))

# 압축
zip_path = os.path.join(result_dir, "seatbelt_yolo_export.zip")
make_archive(zip_path.replace(".zip", ""), 'zip', export_dir)

print("✅ 압축 완료:", zip_path)


📦 압축 대상 총 이미지 수: 10000
✅ 압축 완료: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export/seatbelt_yolo_export.zip


In [12]:
# Cell 5: 다운로드
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
missing_txt = []

for img_path in total_img_files:
    base = os.path.splitext(os.path.basename(img_path))[0]
    txt_path = os.path.join(label_dir, base + ".txt")
    if not os.path.exists(txt_path):
        missing_txt.append(base)

print(f"❌ 누락된 .txt 수: {len(missing_txt)}개")
print("누락된 파일 이름:", missing_txt)


❌ 누락된 .txt 수: 0개
누락된 파일 이름: []


In [ ]:
# 필요 시 후처리만 초기화, 수치 변경 후 재작업

In [ ]:
# 후처리 초기화
import shutil
import os

# 🎯 경로 설정 (필요시 수정)
save_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result"
corrected_label_dir = os.path.join(save_dir, "labels_corrected")
zip_file = os.path.join(save_dir, "seatbelt_yolo_export.zip")
zip_temp_dir = os.path.join(save_dir, "export_zip_temp")

# ✅ 1. labels_corrected 폴더 삭제
if os.path.exists(corrected_label_dir):
    shutil.rmtree(corrected_label_dir)
    print(f"🗑️ 삭제 완료: {corrected_label_dir}")
else:
    print(f"ℹ️ 존재하지 않음: {corrected_label_dir}")

# ✅ 2. zip 파일 삭제
if os.path.exists(zip_file):
    os.remove(zip_file)
    print(f"🗑️ 삭제 완료: {zip_file}")
else:
    print(f"ℹ️ 존재하지 않음: {zip_file}")

# ✅ 3. 임시 압축 폴더 삭제
if os.path.exists(zip_temp_dir):
    shutil.rmtree(zip_temp_dir)
    print(f"🗑️ 삭제 완료: {zip_temp_dir}")
else:
    print(f"ℹ️ 존재하지 않음: {zip_temp_dir}")

ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/labels_corrected
ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/seatbelt_yolo_export.zip
ℹ️ 존재하지 않음: /content/drive/MyDrive/Colab Notebooks/Movon/dataset/inference_result/export_zip_temp
